# Курс Спортивный анализ данных. Платформа Kaggle

# Практическое задание урока 5. Feature Engineering, Feature Selection, part I

Продолжим работу с данными, которые были использованы в ДЗ2 и 3, продолжим решать задачу обнаружения мошеннических транзакций, что позволит получить полное решение задачи / полный пайплайн.

Задание 0: выбрать любую модель машнного обучения и зафиксировать любой тип валидации. Обучить базовую модель и зафиксировать базовое качество модели. В каждом следующем задании нужно будет обучить выбранную модель и оценивать ее качество на зафиксированной схеме валидации. После каждого задания, требуется сделать вывод о достигаемом качестве модели, по сравнению с качестом из предыдущего шага.

Задание 1: признак `TransactionDT` - это смещение в секундах относительно базовой даты. Базовая дата - `2017-12-01`, преобразовать признак `TransactionDT` в `datetime`, прибавив к базовой дате исходное значение признака. Из полученного признака выделить год, месяц, день недели, час, день.

Задание 2: сделать конкатенацию признаков
* `card1` + `card2`;
* `card1` + `card2` + `card_3` + `card_5`;
* `card1` + `card2` + `card_3` + `card_5` + `addr1` + `addr2`

Рассматривать их как категориальных признаки.

Задание 3: Сделать `FrequencyEncoder` для признаков `card1` - `card6`, `addr1`, `addr2`.

Задание 4: Создать признаки на основе отношения: `TransactionAmt` к вычисленной статистике. Статистика - среднее значение / стандартное отклонение `TransactionAmt`, сгруппированное по `card1` - `card6`, `addr1`, `addr2`, и по признакам, созданным в задании 2.

Задание 5: Создать признаки на основе отношения: D15 к вычисленной статистике. Статистика - среднее значение / стандартное отклонение D15, сгруппированное по `card1` - `card6`, `addr1`, `addr2`, и по признакам, созданным в задании 2.

Задание 6: выделить дробную часть и целую часть признака TransactionAmt в два отдельных признака. После создать отдельных признак - логарифм от `TransactionAmt`

Задание 7 (опция): выполнить предварительную подготовку / очистку признаков `P_emaildomain` и `R_emaildomain` (что и как делать - остается на ваше усмотрение) и сделать Frequency Encoding для очищенных признаков.

## Подключение библиотек и скриптов

In [171]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
from scipy.stats import rankdata
from scipy.stats.mstats import winsorize

#from sklearn.ensemble import RandomForestClassifier
#from sklearn.model_selection import cross_val_score
#from sklearn.linear_model import LogisticRegression, LinearRegression
#from sklearn.preprocessing import StandardScaler, MinMaxScaler
#from sklearn.pipeline import Pipeline

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, cross_val_score
import catboost as cb
#from sklearn.preprocessing import LabelEncoder
from pathlib import Path
import math
pd.set_option("max.columns", None)

In [172]:
def catboost_hold_out_validation(params, X, y, split_params = [0.7, 0.2, 0.1], categorical = None, seed = 42):
    """
    Hold-Out валидация для модели catbooost.

    Parameters
    ----------
    params: dict
        Словарь гиперпараметров модели.

    X: pandas.core.frame.DataFrame
        Матрица признако для обучения модели.

    y: pandas.core.frame.Series
        Вектор целевой переменной для обучения модели.

    split_params: List[float], optional, default = [0.7, 0.2, 0.1]
        Параметры (доли) разбиения выборки.
        Опциональный параметр, по умолчанию, равен [0.7, 0.2, 0.1].

    categorical: str, optional, default = None
        Список категориальных признаков.
        Опциональный параметр, по умолчанию, не используется.

    Returns
    -------
    estimator: catboost.core.CatBoostClassifier
        Обученный классификатор catboost.

    test_prediction: np.array, optional
        Вектор прогнозов для тестовой выборки.
        Опциональный объект, возвращается только, если split_params
        содержит 3 значения.

    """
    if categorical is not None:
        X[categorical] = X[categorical].astype(str)

    numeric = list(set(X.columns) - set(categorical))
    x_train, x_valid = train_test_split(
        X, train_size=split_params[0], random_state=seed
    )
    y_train, y_valid = train_test_split(
        y, train_size=split_params[0], random_state=seed
    )

    if len(split_params) == 3:
        test_size = int(split_params[2] * X.shape[0])

        x_valid, x_test = train_test_split(
            x_valid, test_size=test_size, random_state=seed
        )
        y_valid, y_test = train_test_split(
            y_valid, test_size=test_size, random_state=seed
        )

    estimator = cb.CatBoostClassifier(**params)
    estimator.fit(
        x_train, y_train, categorical,
        eval_set=[(x_train, y_train), (x_valid, y_valid)]
    )

    print("="*80)
    train_score = roc_auc_score(y_train, estimator.predict_proba(x_train)[:, 1])
    print(f"Train Score = {round(train_score, 4)}")
    valid_score = roc_auc_score(y_valid, estimator.predict_proba(x_valid)[:, 1])
    print(f"Valid Score = {round(valid_score, 4)}")

    if len(split_params) == 3:

        test_prediction = estimator.predict_proba(x_test)[:, 1]
        test_score = roc_auc_score(y_test, test_prediction)
        print(f"Test Score = {round(test_score, 4)}")

        return estimator, test_prediction

    else:
        return estimator

In [173]:
def get_input(dataset_path: str) -> pd.DataFrame:
    """
    Считывание данных и вывод основной информации о наборе данных.
    
    Parametrs
    ---------
    dataset_path: str
        Название файла
        
    Returns
    -------
    data: pandas.core.frame.DataFrame
        Загруженный набор данных в pandas.DataFrame
    """
    
    data_root = Path('D:/DS_materials/208_kaggle/data/')
    dataset = pd.read_csv(f'{data_root}/{dataset_path}')
    
    print(f"{dataset_path} shape: {dataset.shape[0]} rows, {dataset.shape[1]} cols")

    return dataset

## Загрузка данных

In [174]:
train = get_input("assignment_2_train.csv")
test = get_input("assignment_2_test.csv")

assignment_2_train.csv shape: 180000 rows, 394 cols
assignment_2_test.csv shape: 100001 rows, 394 cols


## Подготовка данных

In [175]:
ID = 'TransactionID'
DATE = 'TransactionDT'
TARGET = 'isFraud'  # Target

#ALL_NUMERICAL_FEATS = train.select_dtypes(include=[np.number]).columns # Numerical + Target
#NUMERICAL_FEATS = ALL_NUMERICAL_FEATS.drop([TARGET, ID, DATE]) # Numerical only
CAT_FEATS = train.dtypes[train.dtypes=="object"].index.tolist()  # Categorical

print(f"Categorical Features Count: {len(CAT_FEATS)}")
#print(f"Numerical Features Count: {len(NUMERICAL_FEATS)}")

Categorical Features Count: 14


## Задание 0: 

* выбрать любую модель машнного обучения и зафиксировать любой тип валидации. 
* Обучить базовую модель и зафиксировать базовое качество модели. 
* В каждом следующем задании нужно будет обучить выбранную модель и оценивать ее качество на зафиксированной схеме валидации. 
* После каждого задания, требуется сделать вывод о достигаемом качестве модели, по сравнению с качестом из предыдущего шага.

**Baseline model**

In [176]:
seed = 42
np.random.seed(seed)

X = train.copy()
X = X.drop([TARGET, ID, DATE], axis=1)
y = train[TARGET]

In [177]:
params_cb = {
    "n_estimators": 2000,
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "task_type": "CPU",
    "max_bin": 20,
    "verbose": 100,
    "max_depth": 6,
    "l2_leaf_reg": 100,
    "early_stopping_rounds": 50,
    "thread_count": 6,
    "random_seed": seed
}

In [178]:
model_cb_0 = catboost_hold_out_validation(params=params_cb,
                                                        X=X,
                                                        y=y,
                                                        split_params =[0.7, 0.3],
                                                        categorical = CAT_FEATS,
                                                        seed=seed)

0:	test: 0.6220983	test1: 0.6208272	best: 0.6208272 (0)	total: 724ms	remaining: 24m 7s
100:	test: 0.8633538	test1: 0.8602173	best: 0.8602173 (100)	total: 1m 10s	remaining: 22m 12s
200:	test: 0.8801031	test1: 0.8761358	best: 0.8761416 (199)	total: 2m 19s	remaining: 20m 51s
300:	test: 0.8891477	test1: 0.8829224	best: 0.8829224 (300)	total: 3m 26s	remaining: 19m 23s
400:	test: 0.8956459	test1: 0.8883050	best: 0.8883050 (400)	total: 4m 34s	remaining: 18m 14s
500:	test: 0.8983561	test1: 0.8906642	best: 0.8906642 (500)	total: 5m 43s	remaining: 17m 8s
600:	test: 0.9002421	test1: 0.8923206	best: 0.8923206 (600)	total: 6m 51s	remaining: 15m 58s
700:	test: 0.9016286	test1: 0.8933351	best: 0.8933351 (700)	total: 8m 3s	remaining: 14m 55s
800:	test: 0.9038404	test1: 0.8949161	best: 0.8949161 (800)	total: 9m 11s	remaining: 13m 44s
900:	test: 0.9062002	test1: 0.8967663	best: 0.8967663 (900)	total: 10m 20s	remaining: 12m 37s
1000:	test: 0.9079216	test1: 0.8981092	best: 0.8981092 (1000)	total: 11m 31s	

**Итого:** 
* Базовое качество модели на валидационной выборке Valid Score = 0.9005

## Задание 1:

* признак `TransactionDT` - это смещение в секундах относительно базовой даты. 
* Базовая дата - `2017-12-01`, преобразовать признак `TransactionDT` в `datetime`, прибавив к базовой дате исходное значение признака. 
* Из полученного признака выделить год, месяц, день недели, час, день.

In [179]:
base_date = pd.to_datetime('2017-12-01')
train[DATE] = pd.to_datetime(train[DATE], unit='s', origin=base_date)

In [180]:
train[DATE + '_Year'] = train[DATE].dt.year
train[DATE + '_Month'] = train[DATE].dt.month
train[DATE + '_WeekDay'] = train[DATE].dt.weekday  # день недели как целое число, где понедельник — 0, а воскресенье — 6
train[DATE + '_Hour'] = train[DATE].dt.hour
train[DATE + '_Day'] = train[DATE].dt.day  # От 1 до количества дней в данном месяце данного года

train.iloc[:, -5:].head()

,TransactionDT_Year,TransactionDT_Month,TransactionDT_WeekDay,TransactionDT_Hour,TransactionDT_Day
0,2017,12,5,0,2
1,2017,12,5,0,2
2,2017,12,5,0,2
3,2017,12,5,0,2
4,2017,12,5,0,2


In [181]:
# Обратное преобразование в timestamp
train[DATE] = (train[DATE].astype('int64') / 10**9).astype('int64')

In [182]:
%%time

X = train.drop([TARGET, ID], axis=1)
y = train[TARGET]

model_cb_1 = catboost_hold_out_validation(params=params_cb,
                                                        X=X,
                                                        y=y,
                                                        split_params =[0.7, 0.3],
                                                        categorical = CAT_FEATS,
                                                        seed=seed)

0:	test: 0.6642880	test1: 0.6650940	best: 0.6650940 (0)	total: 796ms	remaining: 26m 31s
100:	test: 0.8697350	test1: 0.8662081	best: 0.8662081 (100)	total: 1m 12s	remaining: 22m 46s
200:	test: 0.8851765	test1: 0.8811913	best: 0.8811913 (200)	total: 2m 23s	remaining: 21m 22s
300:	test: 0.8947221	test1: 0.8890216	best: 0.8890216 (300)	total: 3m 31s	remaining: 19m 54s
400:	test: 0.9010397	test1: 0.8946221	best: 0.8946221 (400)	total: 4m 41s	remaining: 18m 43s
500:	test: 0.9031435	test1: 0.8961790	best: 0.8961790 (500)	total: 5m 50s	remaining: 17m 28s
600:	test: 0.9050670	test1: 0.8978663	best: 0.8978663 (600)	total: 7m	remaining: 16m 18s
700:	test: 0.9080783	test1: 0.9001840	best: 0.9001840 (700)	total: 8m 11s	remaining: 15m 10s
800:	test: 0.9096685	test1: 0.9015079	best: 0.9015079 (800)	total: 9m 22s	remaining: 14m 1s
900:	test: 0.9115563	test1: 0.9030572	best: 0.9030589 (898)	total: 10m 32s	remaining: 12m 50s
1000:	test: 0.9127549	test1: 0.9039080	best: 0.9039080 (1000)	total: 11m 41s	re

**Итого:** 
* Базовое качество модели на валидационной выборке Valid Score = 0.9005
* На шаге 1 качество модели на валидационной выборке Valid Score = 0.9055, что немного выше, чем базовое качество

## Задание 2: 

сделать конкатенацию признаков
* `card1` + `card2`;
* `card1` + `card2` + `card_3` + `card_5`;
* `card1` + `card2` + `card_3` + `card_5` + `addr1` + `addr2`

Рассматривать их как категориальных признаки.

In [183]:
train[CAT_FEATS].head(2)

,ProductCD,card4,card6,P_emaildomain,R_emaildomain,M1,M2,M3,M4,M5,M6,M7,M8,M9
0,W,discover,credit,NaN,NaN,T,T,T,M2,F,T,NaN,NaN,NaN
1,W,mastercard,credit,gmail.com,NaN,NaN,NaN,NaN,M0,T,T,NaN,NaN,NaN


In [184]:
features = ['card1', 'card2', 'card3', 'card5', 'addr1', 'addr2']

for feature in features:
    train[feature] = train[feature].astype('str')

In [185]:
train["card1_card2"] = train['card1'] + " | " + train['card2']
train["card1_card2_card3_card5"] = train['card1_card2'] + " | " + train['card3'] + " | " + train['card5']
train["card1_card2_card3_card5_addr1_addr2"] = train['card1_card2_card3_card5'] + " | " +  train['addr1'] + " | " + train['addr2']

In [186]:
CAT_FEATS = train.dtypes[train.dtypes=="object"].index.tolist()  # Categorical
train[CAT_FEATS].head(2)

,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,P_emaildomain,R_emaildomain,M1,M2,M3,M4,M5,M6,M7,M8,M9,card1_card2,card1_card2_card3_card5,card1_card2_card3_card5_addr1_addr2
0,W,13926,nan,150.0,discover,142.0,credit,315.0,87.0,NaN,NaN,T,T,T,M2,F,T,NaN,NaN,NaN,13926 | nan,13926 | nan | 150.0 | 142.0,13926 | nan | 150.0 | 142.0 | 315.0 | 87.0
1,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,gmail.com,NaN,NaN,NaN,NaN,M0,T,T,NaN,NaN,NaN,2755 | 404.0,2755 | 404.0 | 150.0 | 102.0,2755 | 404.0 | 150.0 | 102.0 | 325.0 | 87.0


In [187]:
%%time

X = train.drop([TARGET, ID], axis=1)
y = train[TARGET]

model_cb_2 = catboost_hold_out_validation(params=params_cb,
                                                        X=X,
                                                        y=y,
                                                        split_params =[0.7, 0.3],
                                                        categorical = CAT_FEATS,
                                                        seed=seed)

0:	test: 0.5591302	test1: 0.5500489	best: 0.5500489 (0)	total: 840ms	remaining: 28m
100:	test: 0.9575981	test1: 0.9120237	best: 0.9120237 (100)	total: 1m 35s	remaining: 29m 53s
200:	test: 0.9700971	test1: 0.9265068	best: 0.9265068 (200)	total: 3m 13s	remaining: 28m 52s
300:	test: 0.9768168	test1: 0.9366387	best: 0.9366387 (300)	total: 4m 56s	remaining: 27m 52s
400:	test: 0.9786857	test1: 0.9399726	best: 0.9399726 (400)	total: 6m 34s	remaining: 26m 13s
500:	test: 0.9791169	test1: 0.9413302	best: 0.9413302 (498)	total: 8m 10s	remaining: 24m 27s
600:	test: 0.9794137	test1: 0.9416932	best: 0.9417021 (593)	total: 9m 47s	remaining: 22m 46s
700:	test: 0.9802281	test1: 0.9426494	best: 0.9426494 (700)	total: 11m 26s	remaining: 21m 11s
800:	test: 0.9805624	test1: 0.9431072	best: 0.9431173 (793)	total: 13m 2s	remaining: 19m 31s
900:	test: 0.9811050	test1: 0.9437610	best: 0.9437611 (899)	total: 14m 41s	remaining: 17m 54s
1000:	test: 0.9818946	test1: 0.9447020	best: 0.9447088 (999)	total: 16m 16s	r

**Итого:** 
* Базовое качество модели на валидационной выборке Valid Score = 0.9005
* На шаге 1 качество модели на валидационной выборке Valid Score = 0.9055, что немного выше, чем базовое качество
* На шаге 2 качество модели на валидационной выборке Valid Score = 0.9481, что выше, чем качество на шаге 1. При этом уровень переобучения возрастает по сравнению с шагом 1.

## Задание 3: 

Сделать `FrequencyEncoder` для признаков `card1` - `card6`, `addr1`, `addr2`.

In [188]:
features = ['card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2']

for feature in features:
    freq_enc = train[feature].value_counts(normalize=True)
    train[feature + "_freq_enc"] = train[feature].map(freq_enc)

In [189]:
train.iloc[:, -8:].head(2)

,card1_freq_enc,card2_freq_enc,card3_freq_enc,card4_freq_enc,card5_freq_enc,card6_freq_enc,addr1_freq_enc,addr2_freq_enc
0,0.000061,0.014506,0.879722,0.013212,0.000272,0.317951,0.038156,0.876289
1,0.001244,0.006756,0.879722,0.302797,0.054433,0.317951,0.071367,0.876289


In [190]:
%%time

X = train.drop([TARGET, ID], axis=1)
y = train[TARGET]

model_cb_3 = catboost_hold_out_validation(params=params_cb,
                                                        X=X,
                                                        y=y,
                                                        split_params =[0.7, 0.3],
                                                        categorical = CAT_FEATS,
                                                        seed=seed)

0:	test: 0.6864321	test1: 0.6865562	best: 0.6865562 (0)	total: 1.43s	remaining: 47m 47s
100:	test: 0.9585121	test1: 0.9142675	best: 0.9142675 (100)	total: 1m 39s	remaining: 31m 5s
200:	test: 0.9724830	test1: 0.9272972	best: 0.9272972 (200)	total: 3m 17s	remaining: 29m 25s
300:	test: 0.9786306	test1: 0.9375616	best: 0.9375616 (300)	total: 4m 59s	remaining: 28m 10s
400:	test: 0.9803445	test1: 0.9408518	best: 0.9408518 (400)	total: 6m 38s	remaining: 26m 27s
500:	test: 0.9809132	test1: 0.9421707	best: 0.9421707 (499)	total: 8m 16s	remaining: 24m 45s
600:	test: 0.9816468	test1: 0.9432190	best: 0.9432190 (600)	total: 9m 52s	remaining: 22m 58s
700:	test: 0.9821014	test1: 0.9438985	best: 0.9438992 (695)	total: 11m 27s	remaining: 21m 13s
800:	test: 0.9826285	test1: 0.9445068	best: 0.9445068 (800)	total: 13m	remaining: 19m 27s
900:	test: 0.9836870	test1: 0.9458724	best: 0.9458724 (900)	total: 14m 36s	remaining: 17m 49s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.946048055

**Итого:** 
* Базовое качество модели на валидационной выборке Valid Score = 0.9005
* На шаге 1 качество модели на валидационной выборке Valid Score = 0.9055, что немного выше, чем базовое качество
* На шаге 2 качество модели на валидационной выборке Valid Score = 0.9481, что выше, чем качество на шаге 1. При этом уровень переобучения возрастает по сравнению с шагом 1.
* На шаге 3 качество модели на валидационной выборке Valid Score = 0.946, то немного ниже, чем качество на шаге 2. При этом имеется переобучение также как на шаге 2.

## Задание 4: 
* Создать признаки на основе отношения: `TransactionAmt` к вычисленной статистике. 
* Статистика - среднее значение / стандартное отклонение `TransactionAmt`, сгруппированное по `card1` - `card6`, `addr1`, `addr2`, и по признакам, созданным в задании 2.

In [191]:
value_col = "TransactionAmt"
new_features = ["card1_card2", "card1_card2_card3_card5", "card1_card2_card3_card5_addr1_addr2"]
features = features + new_features

In [192]:
data = X.copy()

for feature in features:
    value_col_mean_by_feature = data.groupby(feature, as_index=False)[value_col].mean()
    group_feat_name = value_col + "_mean_by_" + feature
    value_col_mean_by_feature = value_col_mean_by_feature.rename(columns={value_col: group_feat_name})

    data = data.merge(value_col_mean_by_feature, how="left", on=feature)
    
    data[value_col + "_per_" + group_feat_name] = data[value_col] / data[group_feat_name]
    data = data.drop(group_feat_name, axis=1)

In [193]:
data.iloc[:, -len(features):].head(2)

,TransactionAmt_per_TransactionAmt_mean_by_card1,TransactionAmt_per_TransactionAmt_mean_by_card2,TransactionAmt_per_TransactionAmt_mean_by_card3,TransactionAmt_per_TransactionAmt_mean_by_card4,TransactionAmt_per_TransactionAmt_mean_by_card5,TransactionAmt_per_TransactionAmt_mean_by_card6,TransactionAmt_per_TransactionAmt_mean_by_addr1,TransactionAmt_per_TransactionAmt_mean_by_addr2,TransactionAmt_per_TransactionAmt_mean_by_card1_card2,TransactionAmt_per_TransactionAmt_mean_by_card1_card2_card3_card5,TransactionAmt_per_TransactionAmt_mean_by_card1_card2_card3_card5_addr1_addr2
0,0.354505,0.351143,0.488098,0.310646,0.555175,0.403732,0.514610,0.486380,0.231126,0.231126,1.000000
1,0.126313,0.145875,0.206640,0.230124,0.152468,0.170923,0.194195,0.205913,0.126313,0.126313,0.127861


In [194]:
%%time

X = data.copy()
y = train[TARGET]

model_cb_4 = catboost_hold_out_validation(params=params_cb,
                                                        X=X,
                                                        y=y,
                                                        split_params =[0.7, 0.3],
                                                        categorical = CAT_FEATS,
                                                        seed=seed)

0:	test: 0.7436163	test1: 0.7450759	best: 0.7450759 (0)	total: 1.03s	remaining: 34m 26s
100:	test: 0.9599069	test1: 0.9152070	best: 0.9152070 (100)	total: 1m 37s	remaining: 30m 33s
200:	test: 0.9705584	test1: 0.9288680	best: 0.9288680 (200)	total: 3m 15s	remaining: 29m 11s
300:	test: 0.9781034	test1: 0.9374928	best: 0.9375039 (299)	total: 4m 57s	remaining: 27m 58s
400:	test: 0.9813943	test1: 0.9416577	best: 0.9416577 (400)	total: 6m 32s	remaining: 26m 5s
500:	test: 0.9821740	test1: 0.9433824	best: 0.9433824 (500)	total: 8m 2s	remaining: 24m 2s
600:	test: 0.9826830	test1: 0.9445535	best: 0.9445535 (600)	total: 9m 29s	remaining: 22m 4s
700:	test: 0.9827901	test1: 0.9449695	best: 0.9449695 (700)	total: 10m 55s	remaining: 20m 14s
800:	test: 0.9835520	test1: 0.9461430	best: 0.9461430 (800)	total: 12m 25s	remaining: 18m 36s
900:	test: 0.9837784	test1: 0.9464291	best: 0.9464291 (900)	total: 13m 54s	remaining: 16m 58s
1000:	test: 0.9843019	test1: 0.9468290	best: 0.9468301 (999)	total: 15m 21s	

**Итого:** 
* Базовое качество модели на валидационной выборке Valid Score = 0.9005
* На шаге 1 качество модели на валидационной выборке Valid Score = 0.9055, что немного выше, чем базовое качество
* На шаге 2 качество модели на валидационной выборке Valid Score = 0.9481, что выше, чем качество на шаге 1. При этом уровень переобучения возрастает по сравнению с шагом 1.
* На шаге 3 качество модели на валидационной выборке Valid Score = 0.946, то немного ниже, чем качество на шаге 2. При этом имеется переобучение также как на шаге 2.
* На шаге 4 качество модели на валидационной выборке Valid Score = 0.9483, что выше, чем качество на шаге 3 и на предыдущих шагах.

## Задание 5: 
* Создать признаки на основе отношения: D15 к вычисленной статистике. 
* Статистика - среднее значение / стандартное отклонение D15, сгруппированное по `card1` - `card6`, `addr1`, `addr2`, и по признакам, созданным в задании 2.

In [195]:
value_col = "D15"
data = X.copy()
data[value_col].head(5)

0      0.0
1      0.0
2    315.0
3    111.0
4      NaN
Name: D15, dtype: float64

In [196]:
for feature in features:
    value_col_mean_by_feature = data.groupby(feature, as_index=False)[value_col].mean()
    group_feat_name = value_col + "_mean_by_" + feature
    value_col_mean_by_feature = value_col_mean_by_feature.rename(columns={value_col: group_feat_name})

    data = data.merge(value_col_mean_by_feature, how="left", on=feature)
    
    data[value_col + "_per_" + group_feat_name] = data[value_col] / data[group_feat_name]
    data = data.drop(group_feat_name, axis=1)

In [197]:
data.iloc[:, -len(features):].head(5)

,D15_per_D15_mean_by_card1,D15_per_D15_mean_by_card2,D15_per_D15_mean_by_card3,D15_per_D15_mean_by_card4,D15_per_D15_mean_by_card5,D15_per_D15_mean_by_card6,D15_per_D15_mean_by_addr1,D15_per_D15_mean_by_addr2,D15_per_D15_mean_by_card1_card2,D15_per_D15_mean_by_card1_card2_card3_card5,D15_per_D15_mean_by_card1_card2_card3_card5_addr1_addr2
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2.749191,1.675925,1.869807,2.080389,2.483547,1.984550,1.748451,1.857952,2.749191,2.749191,5.965909
3,0.616704,0.642710,0.658884,0.795717,0.616677,0.699318,0.728034,0.654707,0.616704,0.616704,0.660557
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [198]:
%%time

X = data.copy()
y = train[TARGET]

model_cb_5 = catboost_hold_out_validation(params=params_cb,
                                                        X=X,
                                                        y=y,
                                                        split_params =[0.7, 0.3],
                                                        categorical = CAT_FEATS,
                                                        seed=seed)

0:	test: 0.6605096	test1: 0.6716779	best: 0.6716779 (0)	total: 1.55s	remaining: 51m 34s
100:	test: 0.9584288	test1: 0.9129765	best: 0.9129765 (100)	total: 1m 33s	remaining: 29m 9s
200:	test: 0.9680573	test1: 0.9264906	best: 0.9264906 (200)	total: 3m 4s	remaining: 27m 30s
300:	test: 0.9754294	test1: 0.9360598	best: 0.9360598 (300)	total: 4m 40s	remaining: 26m 24s
400:	test: 0.9788973	test1: 0.9400377	best: 0.9400389 (399)	total: 6m 13s	remaining: 24m 49s
500:	test: 0.9802494	test1: 0.9419043	best: 0.9419043 (500)	total: 7m 42s	remaining: 23m 4s
600:	test: 0.9807882	test1: 0.9426677	best: 0.9426811 (599)	total: 9m 13s	remaining: 21m 29s
700:	test: 0.9810294	test1: 0.9430628	best: 0.9430628 (700)	total: 10m 42s	remaining: 19m 49s
800:	test: 0.9815419	test1: 0.9440561	best: 0.9440561 (799)	total: 12m 12s	remaining: 18m 16s
900:	test: 0.9819346	test1: 0.9445689	best: 0.9445689 (900)	total: 13m 38s	remaining: 16m 38s
1000:	test: 0.9825924	test1: 0.9453649	best: 0.9453649 (1000)	total: 15m 5s

**Итого:** 
* Базовое качество модели на валидационной выборке Valid Score = 0.9005
* На шаге 1 качество модели на валидационной выборке Valid Score = 0.9055, что немного выше, чем базовое качество
* На шаге 2 качество модели на валидационной выборке Valid Score = 0.9481, что выше, чем качество на шаге 1. При этом уровень переобучения возрастает по сравнению с шагом 1.
* На шаге 3 качество модели на валидационной выборке Valid Score = 0.946, то немного ниже, чем качество на шаге 2. При этом имеется переобучение также как на шаге 2.
* На шаге 4 качество модели на валидационной выборке Valid Score = 0.9483, что выше, чем качество на шаге 3 и на предыдущих шагах.
* На шаге 5 качество модели на валидационной выборке Valid Score = 0.9476, что немного ниже, чем качество на шаге 4.

## Задание 6: 
* выделить дробную часть и целую часть признака TransactionAmt в два отдельных признака. 
* После создать отдельных признак - логарифм от `TransactionAmt`

In [199]:
data = X.copy()
value_col = "TransactionAmt"
data[value_col].head(5)

0    68.5
1    29.0
2    59.0
3    50.0
4    50.0
Name: TransactionAmt, dtype: float64

In [200]:
import math

In [201]:
data['IntegerPart_of_' + value_col] = data[value_col].apply(lambda x: math.modf(x)[1])
data['FractionalPart_of_' + value_col] = data[value_col].apply(lambda x: math.modf(x)[0])
data['Logarithm_of_' + value_col] = np.log(data[value_col])

data.iloc[:, -3:].head(5)

,IntegerPart_of_TransactionAmt,FractionalPart_of_TransactionAmt,Logarithm_of_TransactionAmt
0,68.0,0.5,4.226834
1,29.0,0.0,3.367296
2,59.0,0.0,4.077537
3,50.0,0.0,3.912023
4,50.0,0.0,3.912023


In [202]:
%%time

X = data.copy()
y = train[TARGET]

model_cb_6 = catboost_hold_out_validation(params=params_cb,
                                                        X=X,
                                                        y=y,
                                                        split_params =[0.7, 0.3],
                                                        categorical = CAT_FEATS,
                                                        seed=seed)

0:	test: 0.5956089	test1: 0.5890069	best: 0.5890069 (0)	total: 1.12s	remaining: 37m 13s
100:	test: 0.9586634	test1: 0.9130151	best: 0.9130151 (100)	total: 1m 26s	remaining: 27m 5s
200:	test: 0.9695288	test1: 0.9275364	best: 0.9275364 (200)	total: 2m 57s	remaining: 26m 25s
300:	test: 0.9764152	test1: 0.9369768	best: 0.9369768 (300)	total: 4m 31s	remaining: 25m 32s
400:	test: 0.9788671	test1: 0.9401976	best: 0.9401976 (400)	total: 6m 3s	remaining: 24m 10s
500:	test: 0.9797111	test1: 0.9414614	best: 0.9414896 (493)	total: 7m 34s	remaining: 22m 40s
600:	test: 0.9806734	test1: 0.9426564	best: 0.9426605 (598)	total: 9m 3s	remaining: 21m 4s
700:	test: 0.9811254	test1: 0.9432546	best: 0.9432551 (699)	total: 10m 31s	remaining: 19m 30s
800:	test: 0.9823769	test1: 0.9447062	best: 0.9447066 (798)	total: 12m 2s	remaining: 18m
900:	test: 0.9830868	test1: 0.9458081	best: 0.9458082 (894)	total: 13m 32s	remaining: 16m 30s
1000:	test: 0.9832524	test1: 0.9460461	best: 0.9460461 (1000)	total: 14m 59s	rema

**Итого:** 
* Базовое качество модели на валидационной выборке Valid Score = 0.9005
* На шаге 1 качество модели на валидационной выборке Valid Score = 0.9055, что немного выше, чем базовое качество
* На шаге 2 качество модели на валидационной выборке Valid Score = 0.9481, что выше, чем качество на шаге 1. При этом уровень переобучения возрастает по сравнению с шагом 1.
* На шаге 3 качество модели на валидационной выборке Valid Score = 0.946, то немного ниже, чем качество на шаге 2. При этом имеется переобучение также как на шаге 2.
* На шаге 4 качество модели на валидационной выборке Valid Score = 0.9483, что выше, чем качество на шаге 3 и на предыдущих шагах.
* На шаге 5 качество модели на валидационной выборке Valid Score = 0.9476, что немного ниже, чем качество на шаге 4.
* На шаге 6 качество модели на валидационной выборке Valid Score = 0.9468, что немного ниже, чем на шаге 5.

В процессе Feature Engineering снижают качество модели генерация частотных признаков и признаки, образованные при дроблении и логарифмировании значений признака `TransactionAmt`. В целом Feature Engineering улучшает качество модели по сравнению с базовым.

## Задание 7 (опция): 
* выполнить предварительную подготовку / очистку признаков `P_emaildomain` и `R_emaildomain` (что и как делать - остается на ваше усмотрение) и сделать Frequency Encoding для очищенных признаков.